In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'cities', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\ProgramData\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by City
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
pd.options.display.max_rows = None

In [9]:
df.rename(columns={'city_name': 'City'}, inplace=True)
df['City'] = df['City'].fillna('Not in City Limits')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,51,"26,397","141,915"
Arlington,100,"3,214","9,281","298,717"
Auburn,"3,328","57,713","148,001","1,598,917"
Bainbridge Island,19,216,"1,140","209,415"
Beaux Arts,0,0,0,460
Bellevue,"22,349","230,018","543,805","3,139,241"
Black Diamond,139,747,"1,242","47,828"
Bonney Lake,"2,247","17,553","30,305","204,445"
Bothell,"11,830","142,760","227,611","1,247,550"


In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,15.7%,84.3%
Arlington,0.0%,1.0%,3.0%,96.0%
Auburn,0.2%,3.2%,8.2%,88.4%
Bainbridge Island,0.0%,0.1%,0.5%,99.3%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.6%,5.8%,13.8%,79.8%
Black Diamond,0.3%,1.5%,2.5%,95.7%
Bonney Lake,0.9%,6.9%,11.9%,80.3%
Bothell,0.7%,8.8%,14.0%,76.5%


#### AM Peak (7 to 8 AM)

In [11]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,1,"5,713","6,542"
Arlington,9,103,631,"22,028"
Auburn,358,"11,344","22,003","98,761"
Bainbridge Island,2,16,149,"15,746"
Beaux Arts,0,0,0,31
Bellevue,"1,856","33,357","58,802","192,263"
Black Diamond,23,70,273,"3,533"
Bonney Lake,332,"2,693","1,917","12,344"
Bothell,451,"28,682","22,085","62,322"


In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,46.6%,53.4%
Arlington,0.0%,0.5%,2.8%,96.7%
Auburn,0.3%,8.6%,16.6%,74.6%
Bainbridge Island,0.0%,0.1%,0.9%,99.0%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.6%,11.7%,20.5%,67.2%
Black Diamond,0.6%,1.8%,7.0%,90.6%
Bonney Lake,1.9%,15.6%,11.1%,71.4%
Bothell,0.4%,25.3%,19.5%,54.9%


#### PM Peak (5 to 6 PM)

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,11,"5,476","7,109"
Arlington,7,306,671,"24,817"
Auburn,318,"9,746","12,896","122,792"
Bainbridge Island,1,24,75,"17,692"
Beaux Arts,0,0,0,38
Bellevue,"5,421","31,568","58,529","218,657"
Black Diamond,23,74,117,"4,116"
Bonney Lake,570,"1,726","3,938","13,977"
Bothell,"4,454","17,365","32,495","73,335"


In [14]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.1%,43.5%,56.4%
Arlington,0.0%,1.2%,2.6%,96.2%
Auburn,0.2%,6.7%,8.8%,84.2%
Bainbridge Island,0.0%,0.1%,0.4%,99.4%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,1.7%,10.0%,18.6%,69.6%
Black Diamond,0.5%,1.7%,2.7%,95.1%
Bonney Lake,2.8%,8.5%,19.5%,69.2%
Bothell,3.5%,13.6%,25.5%,57.5%


# Network Delay
Totals scaled to match network totals. 

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
City,
Algona,498
Arlington,247
Auburn,"4,551"
Bainbridge Island,78
Beaux Arts,0
Bellevue,"17,117"
Black Diamond,37
Bonney Lake,"1,203"
Bothell,"9,726"


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [16]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
City,
Algona,419
Arlington,208
Auburn,"3,829"
Bainbridge Island,66
Beaux Arts,0
Bellevue,"14,400"
Black Diamond,31
Bonney Lake,"1,012"
Bothell,"8,183"
